In [ ]:
# pip install google-cloud-storage
# !pip install google-cloud-documentai google-cloud-storage pandas

In [11]:
import os
from google.cloud import documentai_v1 as documentai
import pandas as pd
from google.cloud import storage

# ✅ No need to set GOOGLE_APPLICATION_CREDENTIALS manually here

# 🔧 Config
project_id = "vercillopersonal"
location = "us"
processor_id = "fe61eee8945a8018"
gcs_input_uri = "gs://vercillo_projects/transactions/amex/2025/2025-06-03.pdf"
output_csv_uri = "gs://vercillo_projects/transactions/amex/exports/2025-06-03.csv"

# 🧠 Document AI client
client = documentai.DocumentProcessorServiceClient()
name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

# ⚙️ Process the document
request = documentai.ProcessRequest(
    name=name,
    skip_human_review=True,
    input_config=documentai.DocumentInputConfig(
        gcs_source=documentai.GcsSource(uri=gcs_input_uri),
        mime_type="application/pdf"
    )
)

result = client.process_document(request=request)
document = result.document

# 📊 Extract fields
entities = []
for entity in document.entities:
    entities.append({
        "type": entity.type_,
        "value": entity.mention_text,
        "confidence": round(entity.confidence, 3),
        "page": entity.page_anchor.page_refs[0].page if entity.page_anchor.page_refs else None
    })

df = pd.DataFrame(entities)

# 💾 Save locally
local_csv = "amex_2025_06_03.csv"
df.to_csv(local_csv, index=False)

# ☁️ Upload to GCS
storage_client = storage.Client()
bucket = storage_client.bucket("vercillo_projects")
blob = bucket.blob("transactions/amex/exports/2025-06-03.csv")
blob.upload_from_filename(local_csv)

print(f"✅ Exported to {output_csv_uri}")


OAuthError: ("Error code invalid_grant: Error connecting to the given credential's issuer.", '{"error":"invalid_grant","error_description":"Error connecting to the given credential\'s issuer."}')